In [12]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix , root_mean_squared_error, mean_squared_error
from sklearn.metrics import roc_curve, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import joblib
import warnings
warnings.filterwarnings(action="ignore")


In [2]:
df=pd.read_csv("../data/new_data.csv")


In [ ]:
df_bin = df.copy()
df_bin['target_bin'] = (df_bin['num'] != 0).astype(int)  # 0 -> 0, else -> 1

X = df_bin.drop(columns=['num','target_bin'], axis=1)
y = df_bin['target_bin']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
                                                    test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Function to print training and testing results
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train == False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [9]:
# Initialize the model with a random state for reproducibility
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

# Print training and testing scores
print_score(tree_clf, X_train, y_train, X_test, y_test, train=True)
print_score(tree_clf, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    329.0  407.0       1.0      736.0         736.0
_______________________________________________
Confusion Matrix: 
 [[329   0]
 [  0 407]]

Test Result:
Accuracy Score: 78.26%
_______________________________________________
CLASSIFICATION REPORT:
                   0           1  accuracy   macro avg  weighted avg
precision   0.791667    0.776786  0.782609    0.784226      0.783417
recall      0.695122    0.852941  0.782609    0.774032      0.782609
f1-score    0.740260    0.813084  0.782609    0.776672      0.780630
support    82.000000  102.000000  0.782609  184.000000    184.000000
_______________________________________________
Conf

In [11]:
# Define the hyperparameters and their respective ranges
params = {
    "criterion":("gini", "entropy"), 
    "splitter":("best", "random"), 
    "max_depth":(list(range(1, 20))), 
    "min_samples_split":[2, 3, 4], 
    "min_samples_leaf":list(range(1, 20)), 
}

# Initialize the model with a random state for reproducibility
tree_clf = DecisionTreeClassifier(random_state=42)

# Initialize the GridSearchCV
tree_cv = GridSearchCV(tree_clf, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=3)

# Train the model
tree_cv.fit(X_train, y_train)

# Retrieve the best hyperparameters
best_params = tree_cv.best_params_
print(f"Best paramters: {best_params})")

# Apply the best parameters and re-train the model
tree_clf = DecisionTreeClassifier(**best_params)
tree_clf.fit(X_train, y_train)

# Print scores after hyperparameter tuning
print_score(tree_clf, X_train, y_train, X_test, y_test, train=True)
print_score(tree_clf, X_train, y_train, X_test, y_test, train=False)

Fitting 3 folds for each of 4332 candidates, totalling 12996 fits
Best paramters: {'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 15, 'min_samples_split': 2, 'splitter': 'best'})
Train Result:
Accuracy Score: 83.42%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.826498    0.840095  0.834239    0.833297      0.834017
recall       0.796353    0.864865  0.834239    0.830609      0.834239
f1-score     0.811146    0.852300  0.834239    0.831723      0.833904
support    329.000000  407.000000  0.834239  736.000000    736.000000
_______________________________________________
Confusion Matrix: 
 [[262  67]
 [ 55 352]]

Test Result:
Accuracy Score: 84.78%
_______________________________________________
CLASSIFICATION REPORT:
                   0           1  accuracy   macro avg  weighted avg
precision   0.864865    0.836364  0.847826    0.850614      0.849065
recall      0

In [8]:

# Define the range of hyperparameters to search over
# Number of trees
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
# Maximum depth of the trees
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# Minimum samples needed to split a node
min_samples_split = [2, 5, 10]
# Minimum samples needed at a leaf node
min_samples_leaf = [1, 2, 4]
# Whether bootstrap samples are used
bootstrap = [True, False]

# Combine into a hyperparameter grid
random_grid = {'n_estimators': n_estimators, 
               'max_depth': max_depth, 
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 
               'bootstrap': bootstrap}

# Instantiate the model
rf_clf = RandomForestClassifier(random_state=42)

# Perform Randomized Search
rf_cv = RandomizedSearchCV(estimator=rf_clf, 
                           scoring='f1', 
                           param_distributions=random_grid,
                           n_iter=100, 
                           cv=3, 
                           verbose=2, 
                           random_state=42, 
                           n_jobs=-1)

rf_cv.fit(X_train, y_train)

# Retrieve the best hyperparameters
rf_best_params = rf_cv.best_params_
print(f"Best paramters: {rf_best_params})")

# Train the Random Forest with the best parameters
rf_clf = RandomForestClassifier(**rf_best_params)
rf_clf.fit(X_train, y_train)

# Evaluate the model with tuned hyperparameters
print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best paramters: {'n_estimators': 600, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 70, 'bootstrap': False})
Train Result:
Accuracy Score: 95.24%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.962264    0.944976  0.952446    0.953620      0.952704
recall       0.930091    0.970516  0.952446    0.950304      0.952446
f1-score     0.945904    0.957576  0.952446    0.951740      0.952358
support    329.000000  407.000000  0.952446  736.000000    736.000000
_______________________________________________
Confusion Matrix: 
 [[306  23]
 [ 12 395]]

Test Result:
Accuracy Score: 84.24%
_______________________________________________
CLASSIFICATION REPORT:
                   0           1  accuracy   macro avg  weighted avg
precision   0.844156    0.841121  0.842391    0.842639      0.842474
recall      0.7926

In [13]:
import joblib

# حفظ الموديل
joblib.dump(rf_clf, '../models/hierarchical_model.pkl')
print("Model saved successfully!")


Model saved successfully!
